# Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) data structure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on a trie;
3. Measure suggestion speed;

![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [134]:
!pip install pygtrie
!pip install tqdm
!pip install pyspellchecker

## 1. Build a trie upon a dataset

### 1.1. [5] Read the dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset. Pass the assert.

In [135]:
import pandas as pd
aol_data = pd.read_csv('aol/user-ct-test-collection-01.txt', sep='\t', header=0)

assert aol_data.shape[0] == 3558411, "Dataset size does not match"

### 1.2. [10] Build a Trie

We want a suggest function to be **non-sensitive to stop words** because we don't want to upset the users if they confuses/omits prepositions. Consider *"public events in Innopolis"* vs *"public events at Innopolis"* or *"public events Innopolis"* - they all mean the same.

Build a Trie based on the dataset, **storing query statistics such as query _frequency_, urls and ranks in the nodes**. Some queries may have no associated urls, others may have multiple ranked urls. Think of the way to store this information.

Pass the asserts.

In [136]:
#preprocessing

aol_data = aol_data.dropna(subset=['Query']) #drop empty queries
aol_data = aol_data.astype({'Query': 'string'}) #convert from object to string
aol_data = aol_data[aol_data.Query.str.len() > 1] #drop single character queries
aol_data['Query'] = aol_data['Query'].str.lower() # make all queries lowercase

aol_data.dtypes

AnonID                int64
Query        string[python]
QueryTime            object
ItemRank            float64
ClickURL             object
dtype: object

In [137]:
import pygtrie
import tqdm

stops = set('a on at of to is from for and with using the in &'.split())

aol_trie = pygtrie.CharTrie()

class TrieNode():
    def __init__(self):
        self.queries = []
        self.urls = []
        self.ranks = []
        self.freq = 1
    def __repr__(self):
        return f'TrieNode(freq={self.freq}, urls={self.urls}, ranks={self.ranks})'

def build_trie(data):
    for row in tqdm.tqdm(data.itertuples(), total=data.shape[0]):
        query, url, rank = row.Query, row.ClickURL, row.ItemRank
        key = ' '.join([word for word in query.split() if word not in stops])
        if not key:
            continue
        if key not in aol_trie:
            aol_trie[key] = TrieNode()
        aol_trie[key].queries.append(query)
        aol_trie[key].urls.append(url)
        aol_trie[key].ranks.append(rank)
        aol_trie[key].freq += 1

build_trie(aol_data)

# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a property of list type. But you can do something different. 
    bag += val.urls
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

100%|██████████| 3452322/3452322 [01:28<00:00, 38926.49it/s]

sample question surveys ~ TrieNode(freq=6, urls=['http://www.surveyconnect.com', 'http://www.custominsight.com', 'http://www.askemployees.com', 'http://www.lg-employers.gov.uk', nan], ranks=[7.0, 4.0, 10.0, 1.0, nan])
sample questions immigration interview ~ TrieNode(freq=2, urls=[nan], ranks=[nan])
sample questions interview ~ TrieNode(freq=2, urls=['http://www.quintcareers.com'], ranks=[1.0])
sample questions family interview ~ TrieNode(freq=4, urls=['http://www.grandparents-day.com', 'http://www.quintcareers.com', 'http://jobsearchtech.about.com'], ranks=[2.0, 5.0, 3.0])
sample questions sociology race ethnicity ~ TrieNode(freq=2, urls=[nan], ranks=[nan])
sample questions biology ~ TrieNode(freq=3, urls=['http://www.utexas.edu', 'http://www.troy.k12.ny.us'], ranks=[3.0, 6.0])
sample questions us citizenship test ~ TrieNode(freq=2, urls=['http://uscis.gov'], ranks=[1.0])
sample questionarie teaching evaluation ~ TrieNode(freq=2, urls=[nan], ranks=[nan])
sample questionnaire teaching 

## 2. [15] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

Pass the asserts.

Question for analysis: What is the empirical threshold for minimal prefix for suggest?

In [138]:
def complete_user_query(query: str, trie, top_k=5) -> list[str]:
    query = query.lower()
    results = []
    try: 
        for key, node in trie.items(query):
            url = max(node.urls, key=node.urls.count)
            rank = node.ranks[node.urls.index(url)]
            freq = node.freq
            queries = max(node.queries, key=node.queries.count)
            results.append((key, url, rank, freq, queries))
    except KeyError:
        return []
    results = sorted(results, key=lambda x: x[3], reverse=True)
    q = [queries for key, url, rank, freq, queries in results[:top_k]]
    f = [freq for key, url, rank, freq, queries in results[:top_k]]
    return (q, f)
 
        
inp = "trie"
print("Query:", inp)
print("Results:")
res, _ = complete_user_query(inp, aol_trie) # i added _ because i return a tuple of queries and frequncies
print(res)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"

assert "boys and girls club of conyers georgia" \
            in complete_user_query("boys girls club conyers", aol_trie, 10)[0], "Should be here" # i added [0] because i return a tuple of queries and frequncies

Query: trie
Results:
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', "tried and truechildren's consignment sale"]


## 3. Measure suggest speed ##

### 3.1. [10] Full Trie test

Check how fast your search is working. Consider changing your code if it takes too long on average.

Sucess criterion:
- there is an average and a standard deviation for **multiple runs** of the given bucket.
- there is an average and a standard deviation for **multiple runs** of naive search in the unindexed dataset.

In [139]:
# complete_user_query() but implemented without a trie (for comparison)
def complete_user_query_without_trie(query, dataset, top_k=5):
    freq = {}
    results, indexed_results = set(), []
    
    for item in dataset.itertuples():
        query = item.Query
        freq[query] = freq.get(query, 0) + 1
        if query.startswith(key) or (' ' + key in query):
            results.add(query)
            
    for result in results:
        indexed_results.append((freq[result], result))
        
    indexed_results = sorted(indexed_results, key=lambda x: -x[0])
    return [q[1] for q in indexed_results][:top_k]

In [140]:
import time
import numpy as np

inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

index_runs_averges = []
index_runs_standard_deviation = []
no_index_runs_averages = []
no_index_runs_standard_deviation = []

index_times = []
no_index_times = []
def time_test(runs):
    for i in range(runs):
        print(f"Run {i+1} Started ... ", end='')
        for inp in inp_queries:
            start = time.time() * 1000 #convert to milliseconds
            _ = complete_user_query(inp, aol_trie)
            end = time.time() * 1000 #convert to milliseconds
            index_times.append(end - start)

            start = time.time() * 1000 #convert to milliseconds
            _ = complete_user_query_without_trie(inp, aol_data)
            end = time.time() * 1000 #convert to milliseconds
            no_index_times.append(end - start)
        print(f"Run {i+1} Ended")
        index_runs_averges.append(np.mean(index_times))
        index_runs_standard_deviation.append(np.var(index_times) ** 0.5)
        no_index_runs_averages.append(np.mean(no_index_times))
        no_index_runs_standard_deviation.append(np.var(no_index_times) ** 0.5)

#each run takes ~50 seconds
num_runs = 10   
time_test(num_runs)

print("\n")
print("-+-+-+-+- With index -+-+-+-+-")
print(f"Avg of Avg times in {num_runs} runs: {np.mean(index_times)}")
print(f"Avg of Standard deviation in {num_runs} runs: {np.var(index_times) ** 0.5}")
print("-+-+-+-+- Without index -+-+-+-+-")
print(f"Avg of Avg times in {num_runs} runs: {np.mean(no_index_times)}")
print(f"Avg of Standard deviation times in {num_runs} runs: {np.var(no_index_times) ** 0.5}")

Run 1 Started ... Run 1 Ended
Run 2 Started ... Run 2 Ended
Run 3 Started ... Run 3 Ended
Run 4 Started ... Run 4 Ended
Run 5 Started ... Run 5 Ended
Run 6 Started ... Run 6 Ended
Run 7 Started ... Run 7 Ended
Run 8 Started ... Run 8 Ended
Run 9 Started ... Run 9 Ended
Run 10 Started ... Run 10 Ended


-+-+-+-+- With index -+-+-+-+-
Avg of Avg times in 10 runs: 33.30709783380682
Avg of Standard deviation in 10 runs: 92.71177075761953
-+-+-+-+- Without index -+-+-+-+-
Avg of Avg times in 10 runs: 4253.428506747159
Avg of Standard deviation times in 10 runs: 88.89297017894665


## 4. [10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

You can use use [pyspellchecker](https://pypi.org/project/pyspellchecker/) `candidates()` call, or use any other spellchecker implementation.

In [141]:
import spellchecker as sc

def complete_user_query_with_spellchecker(query, trie, top_k=5) -> list[str]:
    results = []
    words = query.split()
    spell = sc.SpellChecker()
    for i in range(len(words)):
        for correction in spell.candidates(words[i]):
            if words[i] == correction: # word is already correct
                continue
            words[i] = correction  # try that correction
            res = complete_user_query(' '.join(words), trie, top_k)
            if res:
                results.append(res)
    results = sorted(results, key=lambda x: x[1], reverse=True)
    if results:
        return results[0]
    else: 
        return []
    



In [142]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, aol_trie, 5) == \
            complete_user_query_with_spellchecker(q, aol_trie, 5), "Assert {} and {} give different results".format(q, qc)